In [ ]:
!pip install crewai
# For OpenAI
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4-turbo" # Or gpt-3.5-turbo for a cheaper option
pip install 'crewai[tools]' # Installs the crewai_tools package
pip install crewai_tools # Ensure it's explicitly installed if issues
os.environ["SERPER_API_KEY"] = "YOUR_SERPER_API_KEY"
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Initialize the Serper Tool
serper_tool = SerperDevTool()

# Define Agents
analyzer_agent = Agent(
    role='Medical Diagnostics Expert',
    goal='Accurately interpret and summarize blood test reports, highlighting key indicators and their implications.',
    backstory="A meticulous medical professional with years of experience in analyzing laboratory results and identifying potential health concerns.",
    verbose=True,
    allow_delegation=False, # Agents should be focused on their core task
    llm=llm # Assuming 'llm' is defined from your chosen model
)

researcher_agent = Agent(
    role='Health Article Researcher',
    goal='Find reliable articles and information related to specific health indicators identified in blood tests.',
    backstory="A dedicated research assistant, adept at sifting through vast amounts of information to find the most relevant and credible health resources.",
    verbose=True,
    allow_delegation=True, # Can delegate some sub-tasks if needed (e.g., if a complex search)
    tools=[serper_tool],
    llm=llm
)

recommender_agent = Agent(
    role='Empathetic Health Advisor',
    goal='Synthesize medical information and research findings into clear, actionable health recommendations with supporting links.',
    backstory="An empathetic health coach who translates complex medical information into simple, actionable advice for a healthier lifestyle.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Define Tasks
analyze_task = Task(
    description=(
        "Analyze the provided blood test report. Identify all key markers, their values, and categorize them "
        "as normal, high, or low. For any abnormal markers, briefly explain their potential implications "
        "in an easy-to-understand manner. The output should be a structured summary of the report in markdown format."
        "\n\nBlood Test Report:\n{blood_test_report}"
    ),
    expected_output='A markdown-formatted summary of the blood test, highlighting abnormal findings and their implications.',
    agent=analyzer_agent
)

research_task = Task(
    description=(
        "Based on the abnormal findings and their implications identified in the blood test analysis, "
        "search the internet for reputable articles and information. Focus on common causes, "
        "lifestyle recommendations, and general health advice related to these specific findings. "
        "Prioritize articles from trusted medical sources (.gov, .org, reputable university sites). "
        "Return a markdown-formatted summary of the findings and relevant URLs."
    ),
    expected_output='A markdown-formatted summary of research findings and a list of URLs to relevant articles.',
    agent=researcher_agent,
    context=[analyze_task] # This task depends on the output of analyze_task
)

recommend_task = Task(
    description=(
        "Synthesize the blood test analysis and the research findings. Generate clear, actionable, and "
        "easy-to-understand health recommendations for the individual. Recommendations should cover "
        "lifestyle adjustments (diet, exercise, stress management), potential next steps (e.g., "
        "consulting a doctor), and any other relevant advice. Crucially, provide the links to the "
        "supporting articles found during the research phase, formatted clearly."
    ),
    expected_output='A comprehensive markdown-formatted set of health recommendations with supporting article links.',
    agent=recommender_agent,
    context=[analyze_task, research_task] # This task depends on both previous tasks
)

# Instantiate the Crew
health_crew = Crew(
    agents=[analyzer_agent, researcher_agent, recommender_agent],
    tasks=[analyze_task, research_task, recommend_task],
    process=Process.sequential, # Tasks run in defined order
    verbose=2 # Outputs more detailed information during execution
)

# Example Blood Test Report (replace with actual sample report content)
sample_blood_report = """
Patient Name: John Doe
Date: May 29, 2025

**Complete Blood Count (CBC)**
- White Blood Cell (WBC): 12.5 x 10^9/L (Normal: 4.0-10.0) - HIGH
- Red Blood Cell (RBC): 4.8 x 10^12/L (Normal: 4.5-5.5) - NORMAL
- Hemoglobin (HGB): 14.2 g/dL (Normal: 13.5-17.5) - NORMAL
- Platelets: 280 x 10^9/L (Normal: 150-450) - NORMAL

**Lipid Panel**
- Total Cholesterol: 220 mg/dL (Normal: <200) - HIGH
- LDL Cholesterol: 140 mg/dL (Normal: <100) - HIGH
- HDL Cholesterol: 45 mg/dL (Normal: >60) - LOW
- Triglycerides: 160 mg/dL (Normal: <150) - HIGH

**Glucose**
- Fasting Glucose: 115 mg/dL (Normal: 70-99) - HIGH

**Liver Function Tests**
- ALT: 35 U/L (Normal: 7-56) - NORMAL
- AST: 30 U/L (Normal: 8-40) - NORMAL

**Kidney Function Tests**
- Creatinine: 0.9 mg/dL (Normal: 0.7-1.3) - NORMAL
"""

# Run the Crew
print("### CrewAI Health Analysis Initiated ###")
result = health_crew.kickoff(inputs={'blood_test_report': sample_blood_report})
print("\n\n### CrewAI Health Analysis Completed ###")
print(result)